# Chapter 12: Inheritance: For Good or For Worse

This chapter is about inheritance and subclassing, discussing good and bad practices when building class hierarchies. Chapter 12 emphasises two particulars very specific to Python:

* The pitfalls of subclassing from built-in types.

* Multiple inheritance and the method resolution order.

Multiple inheritance will be illustrated using two important Python projects: the TKinter GUI toolkit and the Django Web framework.

## Subclassing Built-In Types is Tricky

Since Python 2.2 it has been possible to subclass built-in types such as `list` or `dict`. The is **one major caveat**: the code of the built-ins (written in C) does not all special methods overriden by user-defined classes.

In [1]:
# Example 12-1. __setitem__ override is ignored by __init__ and __update__

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

In [2]:
dd["two"] = 2
dd

In [3]:
dd.update(three=3)
dd

#### Things to note about the above implementation:

* `[]` calls our `__setitem__` and works as expected.

* The `update` method from `dict` does not use our version of `__setitem__`.

* This built-in behaviour violates a basic rule of OOP: the search for methods should always start from the class of the target instance (`self`), even when the call happens inside a method implemented in a superclass.

**Takeaway:** Subclassing built-ins is error-prone. Instead, derive classes from `collections` module using `UserDict`, `UserList`, and `UserString` which are designed to be easily extended.

In [4]:
# Example 12-3. DoppelDict now works as expected because it extends UserDict and not dict

import collections

class DoppelDict(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

In [5]:
dd["two"] = 2
dd

In [6]:
dd.update(three=3)
dd

## Multiple Inheritance and Method Resolution Order

Any language implementing multiple inheritance needs to deal with potential naming conflicts when unrelated ancestor classes implement a method by the same name. This is called the **"diamond problem"**.

![](../figs/uml-diamondproblem.png)

In [19]:
# Example 12-4. classes A, B, C, and D for the graph in the above figure

class A:
    def ping(self):
        print("ping", self)
        
class B(A):
    def pong(self):
        print("pong", self)
        
class C(A):
    def pong(self):
        print("PONG", self)
        
class D(B, C):
    def ping(self):
        super().ping()
        print("post-ping", self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)
        

In [9]:
d = D()
d.pong()

pong <__main__.D object at 0x103f442e0>


In [11]:
C.pong(d)

PONG <__main__.D object at 0x103f442e0>


#### Things to note about the above implementation

* Both `B` and `C` implement a `pong` method.

* You can always call a method on a superclass directly, passing the instance as an explicit argument (`C.pong(d)`).

* The ambiguity of the call `d.pong()`(which `pong()` method does it call, `B` or `C`?) is resolved because Python follows a specific order when traversing the inheritance graph.

* The order is called **MRO (method resolution order)**. This is stored in the `__mro__` special method, which holds a tuple of references to the superclasses in MRO order.

In [15]:
repr(D.__mro__)

"(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)"

The recommended way to delegate method calls to superclasses is the `super()` built-in function.

Sometimes it is convenient to bypass the MRO and invoke a method on a superclass directly. In this case, `self` must be passed explicitly since we are accessing an *unbound method*.

In [17]:
d = D()
d.ping()

ping <__main__.D object at 0x103f5b400>
post-ping <__main__.D object at 0x103f5b400>


`ping` of `D` makes two calls:

1. `super().ping()`, which delegates the call to class `A`.

2. `print("post-ping", self)` which outputs this line.

In [20]:
d = D()
d.pingpong()

ping <__main__.D object at 0x103f5ba00>
post-ping <__main__.D object at 0x103f5ba00>
ping <__main__.D object at 0x103f5ba00>
pong <__main__.D object at 0x103f5ba00>
pong <__main__.D object at 0x103f5ba00>
PONG <__main__.D object at 0x103f5ba00>


`pingpong` of `D` makes five calls:

1. `self.ping()` which runs `ping` method of `D`.

2. `super().ping()` which bypasses `ping` in `D` and delegates to `A`.

3. `self.pong()` which finds the `B` implementation of `pong`, according to `__mro__`.

4. `super().pong()` which finds the `B.pong` implementation, also following `__mro__`.

5. `C.pong(self)`, which finds `C.pong`, ignoring `__mro__`.

The MRO not only takes into account the inheritance graph, but also the order in which superclasses are listed in a subclass declaration.

If the `D` class where declared `D(C, B)`, the `__mro__` of class `D` would be different: `C` would be searched before `B`.

## Multiple Inheritance in the Real World

TODO.